# జనరేటివ్ నెట్‌వర్క్స్

రీకరెంట్ న్యూరల్ నెట్‌వర్క్స్ (RNNs) మరియు వాటి గేటెడ్ సెల్ వేరియంట్లు, ఉదాహరణకు లాంగ్ షార్ట్ టర్మ్ మెమరీ సెల్స్ (LSTMs) మరియు గేటెడ్ రీకరెంట్ యూనిట్స్ (GRUs), భాషా మోడలింగ్ కోసం ఒక యంత్రాంగాన్ని అందించాయి, అంటే అవి పదాల క్రమాన్ని నేర్చుకుని, ఒక సీక్వెన్స్‌లో తదుపరి పదం కోసం అంచనాలు ఇవ్వగలవు. ఇది RNNలను **జనరేటివ్ పనుల** కోసం ఉపయోగించడానికి వీలు కల్పిస్తుంది, ఉదాహరణకు సాధారణ టెక్స్ట్ జనరేషన్, మెషీన్ అనువాదం, మరియు ఇమేజ్ క్యాప్షనింగ్ కూడా.

మునుపటి యూనిట్‌లో చర్చించిన RNN నిర్మాణంలో, ప్రతి RNN యూనిట్ తదుపరి హిడెన్ స్టేట్‌ను అవుట్‌పుట్‌గా ఉత్పత్తి చేసింది. అయితే, ప్రతి రీకరెంట్ యూనిట్‌కు మరో అవుట్‌పుట్‌ను కూడా జోడించవచ్చు, ఇది మాకు ఒక **సీక్వెన్స్** (మూల సీక్వెన్స్ పొడవుతో సమానం) అవుట్‌పుట్ ఇవ్వడానికి అనుమతిస్తుంది. అంతేకాకుండా, ప్రతి దశలో ఇన్‌పుట్‌ను స్వీకరించని RNN యూనిట్లను ఉపయోగించవచ్చు, అవి కొంత ప్రారంభ స్థితి వెక్టర్‌ను తీసుకుని, ఆ తర్వాత అవుట్‌పుట్‌ల సీక్వెన్స్‌ను ఉత్పత్తి చేస్తాయి.

ఈ నోట్‌బుక్‌లో, మనం సులభమైన జనరేటివ్ మోడల్స్‌పై దృష్టి సారించబోతున్నాము, ఇవి మనకు టెక్స్ట్‌ను ఉత్పత్తి చేయడంలో సహాయపడతాయి. సౌలభ్యం కోసం, మనం **అక్షర స్థాయి నెట్‌వర్క్**ను నిర్మిద్దాం, ఇది అక్షరాలవారీగా టెక్స్ట్‌ను ఉత్పత్తి చేస్తుంది. శిక్షణ సమయంలో, మనం కొంత టెక్స్ట్ కార్పస్ తీసుకుని, దాన్ని అక్షరాల సీక్వెన్స్‌లుగా విభజించాలి.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## అక్షర పదకోశం నిర్మాణం

అక్షర స్థాయి జనరేటివ్ నెట్‌వర్క్‌ను నిర్మించడానికి, మేము టెక్స్ట్‌ను పదాల స్థానంలో వ్యక్తిగత అక్షరాలుగా విభజించాలి. మునుపటి `TextVectorization` లేయర్ ఇది చేయలేకపోతుంది, కాబట్టి మాకు రెండు ఎంపికలు ఉన్నాయి:

* టెక్స్ట్‌ను మానవీయంగా లోడ్ చేసి 'చేతితో' టోకనైజేషన్ చేయడం, [ఈ అధికారిక Keras ఉదాహరణ](https://keras.io/examples/generative/lstm_character_level_text_generation/)లో చూపినట్లుగా
* అక్షర స్థాయి టోకనైజేషన్ కోసం `Tokenizer` క్లాస్ ఉపయోగించడం.

మేము రెండవ ఎంపికను ఎంచుకుంటాము. `Tokenizer` ను పదాలుగా కూడా టోకనైజ్ చేయడానికి ఉపయోగించవచ్చు, కాబట్టి అక్షర స్థాయి నుండి పద స్థాయి టోకనైజేషన్‌కు సులభంగా మారవచ్చు.

అక్షర స్థాయి టోకనైజేషన్ చేయడానికి, `char_level=True` పారామీటర్‌ను ఇవ్వాలి:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

మేము కూడా ఒక ప్రత్యేక టోకెన్‌ను **సీక్వెన్స్ ముగింపు** సూచించడానికి ఉపయోగించాలనుకుంటున్నాము, దీన్ని మేము `<eos>` అని పిలుస్తాము. దీన్ని మనం చేతితో వాక్యకోశంలో చేర్చుదాం:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

ఇప్పుడు, టెక్స్ట్‌ను సంఖ్యల సీక్వెన్స్‌లుగా ఎన్‌కోడ్ చేయడానికి, మనం ఉపయోగించవచ్చు:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## శీర్షికలు సృష్టించడానికి జనరేటివ్ RNN శిక్షణ

మేము వార్తా శీర్షికలను సృష్టించడానికి RNN ను శిక్షణ ఇవ్వడానికి విధానం ఈ విధంగా ఉంటుంది. ప్రతి దశలో, ఒక శీర్షిక తీసుకుని దాన్ని RNN లో ఇన్పుట్ గా ఇస్తాము, మరియు ప్రతి ఇన్పుట్ అక్షరానికి తర్వాతి అవుట్పుట్ అక్షరాన్ని నెట్‌వర్క్ సృష్టించాలని అడుగుతాము:

!['HELLO' అనే పదం RNN ద్వారా సృష్టించబడుతున్న ఉదాహరణను చూపించే చిత్రం.](../../../../../translated_images/te/rnn-generate.56c54afb52f9781d.webp)

మన సీక్వెన్స్ చివరి అక్షరానికి, నెట్‌వర్క్ `<eos>` టోకెన్ సృష్టించాలని అడుగుతాము.

ఇక్కడ ఉపయోగిస్తున్న జనరేటివ్ RNN ప్రధాన తేడా ఏమిటంటే, మేము RNN యొక్క ప్రతి దశ నుండి అవుట్పుట్ తీసుకుంటాము, కేవలం చివరి సెల్ నుండి మాత్రమే కాదు. ఇది RNN సెల్ కు `return_sequences` పారామీటర్ ను సెట్ చేయడం ద్వారా సాధ్యమవుతుంది.

కాబట్టి, శిక్షణ సమయంలో, నెట్‌వర్క్ కు ఇన్పుట్ ఒక నిర్దిష్ట పొడవు గల ఎన్‌కోడ్ చేసిన అక్షరాల సీక్వెన్స్ ఉంటుంది, అవుట్పుట్ అదే పొడవు గల సీక్వెన్స్ కానీ ఒక ఎలిమెంట్ ముందుకు షిఫ్ట్ చేయబడినది మరియు `<eos>` తో ముగిసినది. మినీబ్యాచ్ అనేది ఇలాంటి అనేక సీక్వెన్స్‌లతో ఉంటుంది, అందువల్ల అన్ని సీక్వెన్స్‌లను సరిపోల్చడానికి **padding** అవసరం.

మనం డేటాసెట్ ను మార్చే ఫంక్షన్లు సృష్టిద్దాం. మినీబ్యాచ్ స్థాయిలో సీక్వెన్స్‌లను ప్యాడ్ చేయాలనుకుంటున్నందున, ముందుగా `.batch()` పిలిచి డేటాసెట్ ను బ్యాచ్ చేస్తాము, ఆపై `map` ద్వారా ట్రాన్స్‌ఫర్మేషన్ చేస్తాము. కాబట్టి, ట్రాన్స్‌ఫర్మేషన్ ఫంక్షన్ మొత్తం మినీబ్యాచ్ ను పారామీటర్ గా తీసుకుంటుంది:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

మేము ఇక్కడ చేసే కొన్ని ముఖ్యమైన విషయాలు:
* మొదట స్ట్రింగ్ టెన్సర్ నుండి అసలు టెక్స్ట్‌ను తీసుకుంటాము
* `text_to_sequences` స్ట్రింగుల జాబితాను ఇంటిజర్ టెన్సర్ల జాబితాగా మార్చుతుంది
* `pad_sequences` ఆ టెన్సర్లను వారి గరిష్ట పొడవుకు ప్యాడ్ చేస్తుంది
* చివరగా, మేము అన్ని అక్షరాలను వన్-హాట్ ఎన్‌కోడ్ చేస్తాము, అలాగే షిఫ్టింగ్ మరియు `<eos>` జోడింపును కూడా చేస్తాము. వన్-హాట్ ఎన్‌కోడ్ చేసిన అక్షరాలు ఎందుకు అవసరమో త్వరలో చూద్దాం

కానీ, ఈ ఫంక్షన్ **Pythonic** గా ఉంటుంది, అంటే దీన్ని ఆటోమేటిక్‌గా Tensorflow కంప్యూటేషనల్ గ్రాఫ్‌గా మార్చలేము. ఈ ఫంక్షన్‌ను నేరుగా `Dataset.map` ఫంక్షన్‌లో ఉపయోగిస్తే ఎర్రర్లు వస్తాయి. అందుకే ఈ Pythonic కాల్‌ను `py_function` రాపర్ ఉపయోగించి చుట్టుకోవాలి:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **గమనిక**: Pythonic మరియు Tensorflow ట్రాన్స్‌ఫర్మేషన్ ఫంక్షన్ల మధ్య తేడా కొంచెం క్లిష్టంగా అనిపించవచ్చు, మరియు మీరు dataset ను `fit` కు పంపించే ముందు సాధారణ Python ఫంక్షన్లతో ట్రాన్స్‌ఫర్మ్ చేయకపోవడం ఎందుకు అనుకుంటున్నారో ఉండవచ్చు. ఇది ఖచ్చితంగా చేయవచ్చు, కానీ `Dataset.map` ఉపయోగించడం పెద్ద లాభం కలిగిస్తుంది, ఎందుకంటే డేటా ట్రాన్స్‌ఫర్మేషన్ పైప్‌లైన్ Tensorflow కంప్యూటేషనల్ గ్రాఫ్ ఉపయోగించి అమలు చేయబడుతుంది, ఇది GPU కంప్యూటేషన్లను ఉపయోగించి CPU/GPU మధ్య డేటా మార్పిడి అవసరాన్ని తగ్గిస్తుంది.

ఇప్పుడు మనం మన జనరేటర్ నెట్‌వర్క్‌ను నిర్మించి శిక్షణ ప్రారంభించవచ్చు. ఇది మునుపటి యూనిట్‌లో చర్చించిన ఏదైనా రికరెంట్ సెల్ (సింపుల్, LSTM లేదా GRU) ఆధారంగా ఉండవచ్చు. మన ఉదాహరణలో మనం LSTM ఉపయోగిస్తాము.

నెట్‌వర్క్ అక్షరాలను ఇన్‌పుట్‌గా తీసుకుంటుంది, మరియు వోకాబ్యులరీ పరిమాణం చాలా చిన్నది కాబట్టి, embedding లేయర్ అవసరం లేదు, one-hot-encoded ఇన్‌పుట్ నేరుగా LSTM సెల్‌లోకి పోవచ్చు. అవుట్‌పుట్ లేయర్ `Dense` క్లాసిఫైయర్ అవుతుంది, ఇది LSTM అవుట్‌పుట్‌ను one-hot-encoded టోకెన్ నంబర్లుగా మార్చుతుంది.

అదనంగా, మనం వేరియబుల్-లెంగ్త్ సీక్వెన్సులతో పని చేస్తున్నందున, `Masking` లేయర్ ఉపయోగించి స్ట్రింగ్‌లో ప్యాడ్ చేసిన భాగాన్ని నిర్లక్ష్యం చేసే మాస్క్ సృష్టించవచ్చు. ఇది తప్పనిసరి కాదు, ఎందుకంటే మనం `<eos>` టోకెన్ తర్వాతి అన్ని విషయాలలో ఎక్కువగా ఆసక్తి చూపడం లేదు, కానీ ఈ లేయర్ రకం గురించి కొంత అనుభవం పొందడానికి దీన్ని ఉపయోగిస్తాము. `input_shape` `(None, vocab_size)` ఉంటుంది, ఇక్కడ `None` వేరియబుల్ లెంగ్త్ సీక్వెన్స్‌ను సూచిస్తుంది, మరియు అవుట్‌పుట్ ఆకారం కూడా `(None, vocab_size)` ఉంటుంది, మీరు `summary` నుండి చూడవచ్చు:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## అవుట్పుట్ ఉత్పత్తి

మనం మోడల్‌ను శిక్షణ ఇచ్చిన తర్వాత, దాన్ని ఉపయోగించి కొంత అవుట్పుట్ ఉత్పత్తి చేయాలనుకుంటున్నాము. ముందుగా, టోకెన్ సంఖ్యల శ్రేణి ద్వారా ప్రాతినిధ్యం వహించిన టెక్స్ట్‌ను డీకోడ్ చేయడానికి ఒక మార్గం అవసరం. దీని కోసం, మనం `tokenizer.sequences_to_texts` ఫంక్షన్‌ను ఉపయోగించవచ్చు; అయితే, ఇది క్యారెక్టర్-స్థాయి టోకనైజేషన్‌తో బాగా పనిచేయదు. అందుకే, మనం tokenizer నుండి టోకెన్ల డిక్షనరీ (దాన్ని `word_index` అంటారు) తీసుకుని, ఒక రివర్స్ మ్యాప్ తయారు చేసి, మన స్వంత డీకోడింగ్ ఫంక్షన్ రాస్తాము:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

ఇప్పుడు, జనరేషన్ చేద్దాం. మనం కొన్ని స్ట్రింగ్ `start` తో ప్రారంభించి, దాన్ని సీక్వెన్స్ `inp` గా ఎన్‌కోడ్ చేస్తాము, ఆపై ప్రతి దశలో మన నెట్‌వర్క్‌ను పిలిచి తదుపరి అక్షరాన్ని అంచనా వేస్తాము.

నెట్‌వర్క్ అవుట్‌పుట్ `out` అనేది `vocab_size` ఎలిమెంట్ల వెక్టర్, ఇది ప్రతి టోకెన్ యొక్క సంభావ్యతలను సూచిస్తుంది, మరియు `argmax` ఉపయోగించి అత్యంత సంభావ్యమైన టోకెన్ నంబర్‌ను కనుగొంటాము. ఆ తర్వాత ఆ అక్షరాన్ని జనరేట్ చేసిన టోకెన్ల జాబితాలో జోడించి, జనరేషన్ కొనసాగిస్తాము. ఒక అక్షరాన్ని జనరేట్ చేసే ఈ ప్రక్రియను అవసరమైన అక్షరాల సంఖ్య `size` సార్లు పునరావృతం చేస్తాము, మరియు `eos_token` కనిపించినప్పుడు ముందుగానే ముగిస్తాము.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## శిక్షణ సమయంలో నమూనా అవుట్పుట్

మాకు *ఖచ్చితత్వం* వంటి ఉపయోగకరమైన మెట్రిక్స్ లేవు కాబట్టి, మా మోడల్ మెరుగుపడుతున్నదని చూడగల唯一 మార్గం శిక్షణ సమయంలో ఉత్పత్తి చేసిన స్ట్రింగ్‌ను **నమూనా** చేయడం మాత్రమే. దీని కోసం, మేము **కాల్‌బ్యాక్‌లు** ఉపయోగిస్తాము, అంటే `fit` ఫంక్షన్‌కు పంపగల ఫంక్షన్లు, ఇవి శిక్షణ సమయంలో నిరంతరం పిలవబడతాయి.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


ఈ ఉదాహరణ ఇప్పటికే కొంత మంచి టెక్స్ట్‌ను ఉత్పత్తి చేస్తోంది, కానీ దీన్ని మరింత మెరుగుపరచవచ్చు కొన్ని మార్గాల్లో:
* **మరింత టెక్స్ట్**. మనం మా పనికి కేవలం శీర్షికలను మాత్రమే ఉపయోగించాము, కానీ మీరు పూర్తి టెక్స్ట్‌తో ప్రయోగాలు చేయాలనుకోవచ్చు. RNNలు పొడవైన సీక్వెన్సులను నిర్వహించడంలో అంతగా బాగుండవు, కాబట్టి వాటిని చిన్న వాక్యాలుగా విభజించడం లేదా ఎప్పుడూ ఒక నిర్దిష్ట సీక్వెన్స్ పొడవైన `num_chars` (ఉదాహరణకు, 256) పై శిక్షణ ఇవ్వడం మంచిది. పై ఉదాహరణను ఇలాంటి నిర్మాణంగా మార్చడానికి మీరు [అధికారిక Keras ట్యుటోరియల్](https://keras.io/examples/generative/lstm_character_level_text_generation/) ను ప్రేరణగా తీసుకోవచ్చు.
* **బహుళస్థాయి LSTM**. 2 లేదా 3 LSTM లేయర్లను ప్రయత్నించడం మంచిది. మునుపటి యూనిట్‌లో చెప్పినట్లుగా, ప్రతి LSTM లేయర్ టెక్స్ట్ నుండి నిర్దిష్ట నమూనాలను తీసుకుంటుంది, మరియు క్యారెక్టర్-స్థాయి జనరేటర్ సందర్భంలో తక్కువ LSTM స్థాయి సిలబుల్స్‌ను గుర్తించడానికి బాధ్యత వహిస్తుందని, మరియు ఉన్నత స్థాయిలు పదాలు మరియు పద సంయోజనాల కోసం ఉంటాయని ఆశించవచ్చు. ఇది LSTM కన్‌స్ట్రక్టర్‌కు లేయర్ల సంఖ్య పారామీటర్‌ను అందించడం ద్వారా సులభంగా అమలు చేయవచ్చు.
* మీరు **GRU యూనిట్లతో** కూడా ప్రయోగాలు చేయాలనుకోవచ్చు మరియు ఏవి మెరుగ్గా పనిచేస్తాయో చూడవచ్చు, అలాగే **విభిన్న హిడెన్ లేయర్ పరిమాణాలతో** కూడా ప్రయత్నించవచ్చు. చాలా పెద్ద హిడెన్ లేయర్ ఓవర్‌ఫిట్టింగ్‌కు దారితీయవచ్చు (ఉదా: నెట్‌వర్క్ ఖచ్చితమైన టెక్స్ట్ నేర్చుకుంటుంది), మరియు చిన్న పరిమాణం మంచి ఫలితాన్ని ఇవ్వకపోవచ్చు.


## సాఫ్ట్ టెక్స్ట్ జనరేషన్ మరియు టెంపరేచర్

మునుపటి `generate` నిర్వచనంలో, మేము ఎప్పుడూ అత్యధిక probability ఉన్న అక్షరాన్ని తదుపరి అక్షరంగా తీసుకుంటున్నాము. దీని వల్ల టెక్స్ట్ తరచుగా అదే అక్షరాల సీక్వెన్స్‌ల మధ్య "సైకిల్" అవుతుంది, ఈ ఉదాహరణలో కనిపించేలా:
```
today of the second the company and a second the company ...
```

కానీ, తదుపరి అక్షరానికి probability distribution చూస్తే, కొన్ని అత్యధిక probabilities మధ్య తేడా పెద్దది కాకపోవచ్చు, ఉదాహరణకు ఒక అక్షరానికి probability 0.2 ఉండొచ్చు, మరొకదానికి 0.19 ఉండొచ్చు. ఉదాహరణకు '*play*' సీక్వెన్స్‌లో తదుపరి అక్షరం space లేదా **e** (పదం *player* లో ఉన్నట్లు) రెండూ సమానంగా ఉండొచ్చు.

దీని ద్వారా మనం తెలుసుకోవాలి, ఎప్పుడూ ఎక్కువ probability ఉన్న అక్షరాన్ని ఎంచుకోవడం "న్యాయం" కాదు, ఎందుకంటే రెండవ అత్యధిక probability ఉన్న అక్షరాన్ని ఎంచుకున్నా అర్థవంతమైన టెక్స్ట్ రావచ్చు. కాబట్టి నెట్‌వర్క్ అవుట్‌పుట్ ఇచ్చిన probability distribution నుండి అక్షరాలను **సాంపిల్** చేయడం మంచిది.

ఈ సాంప్లింగ్ `np.multinomial` ఫంక్షన్ ఉపయోగించి చేయవచ్చు, ఇది **మల్టినోమియల్ డిస్ట్రిబ్యూషన్**ని అమలు చేస్తుంది. ఈ **సాఫ్ట్** టెక్స్ట్ జనరేషన్‌ను అమలు చేసే ఫంక్షన్ క్రింద నిర్వచించబడింది:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

మేము **తాపన** అనే మరో పారామితిని పరిచయం చేసాము, ఇది మేము అత్యధిక సంభావ్యతకు ఎంత కట్టుబడి ఉండాలో సూచించడానికి ఉపయోగిస్తారు. తాపన 1.0 అయితే, మేము సరైన మల్టినోమియల్ శాంప్లింగ్ చేస్తాము, మరియు తాపన అనంతానికి చేరినప్పుడు - అన్ని సంభావ్యతలు సమానంగా మారతాయి, మరియు మేము యాదృచ్ఛికంగా తదుపరి అక్షరాన్ని ఎంచుకుంటాము. క్రింది ఉదాహరణలో, తాపనను చాలా పెంచినప్పుడు టెక్స్ట్ అర్థరహితంగా మారిపోతుందని, మరియు అది 0కి దగ్గరగా ఉన్నప్పుడు "సైకిల్డ్" కఠినంగా ఉత్పత్తి చేసిన టెక్స్ట్ లాగా కనిపిస్తుందని మనం గమనించవచ్చు.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. మూల పత్రం దాని స్వదేశీ భాషలో అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం సిఫార్సు చేయబడుతుంది. ఈ అనువాదం వాడకంలో ఏర్పడిన ఏవైనా అపార్థాలు లేదా తప్పుదారితీసే అర్థాలు కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
